# MNIST 필기체 분류 - LeNet-5 CNN 모델

이 노트북에서는 LeNet-5 아키텍처를 사용하여 MNIST 필기체 숫자를 분류하는 CNN 모델을 구현합니다.

## LeNet-5 아키텍처 개요
![](./lenet-5.png)


In [ ]:
# 필요한 라이브러리 import
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

print(f"TensorFlow 버전: {tf.__version__}")
print(f"Keras 버전: {keras.__version__}")

# GPU 사용 가능 여부 확인
print(f"GPU 사용 가능: {tf.config.list_physical_devices('GPU')}")

# 재현 가능한 결과를 위한 시드 설정
tf.random.set_seed(42)
np.random.seed(42)


In [ ]:
# MNIST 데이터셋 로드
print("MNIST 데이터셋 로딩 중...")
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

print(f"훈련 데이터 형태: {x_train.shape}")
print(f"훈련 레이블 형태: {y_train.shape}")
print(f"테스트 데이터 형태: {x_test.shape}")
print(f"테스트 레이블 형태: {y_test.shape}")

# 데이터 타입 확인
print(f"픽셀 값 범위: {x_train.min()} ~ {x_train.max()}")
print(f"레이블 클래스: {np.unique(y_train)}")


In [ ]:
# 샘플 이미지 시각화
plt.figure(figsize=(12, 8))
for i in range(16):
    plt.subplot(4, 4, i + 1)
    plt.imshow(x_train[i], cmap='gray')
    plt.title(f'레이블: {y_train[i]}')
    plt.axis('off')
plt.suptitle('MNIST 훈련 데이터 샘플', fontsize=16)
plt.tight_layout()
plt.show()

# 클래스 분포 확인
plt.figure(figsize=(10, 6))
unique, counts = np.unique(y_train, return_counts=True)
plt.bar(unique, counts)
plt.title('MNIST 훈련 데이터의 클래스 분포')
plt.xlabel('숫자 클래스')
plt.ylabel('샘플 개수')
plt.xticks(unique)
for i, count in enumerate(counts):
    plt.text(i, count + 100, str(count), ha='center')
plt.show()


In [ ]:
# 데이터 전처리
print("데이터 전처리 중...")

# 1. 정규화 (0-255 → 0-1)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 2. 28x28 → 32x32로 패딩 (LeNet-5는 32x32 입력을 사용)
x_train = tf.pad(x_train, [[0, 0], [2, 2], [2, 2]], mode='CONSTANT')
x_test = tf.pad(x_test, [[0, 0], [2, 2], [2, 2]], mode='CONSTANT')

# 3. 채널 차원 추가 (grayscale)
x_train = tf.expand_dims(x_train, axis=-1)
x_test = tf.expand_dims(x_test, axis=-1)

# 4. 레이블을 원-핫 인코딩
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

print(f"전처리 후 훈련 데이터 형태: {x_train.shape}")
print(f"전처리 후 테스트 데이터 형태: {x_test.shape}")
print(f"전처리 후 훈련 레이블 형태: {y_train.shape}")
print(f"전처리 후 테스트 레이블 형태: {y_test.shape}")
print(f"픽셀 값 범위: {x_train.numpy().min():.3f} ~ {x_train.numpy().max():.3f}")


In [ ]:
# 전처리된 이미지 확인
plt.figure(figsize=(12, 4))
for i in range(4):
    plt.subplot(1, 4, i + 1)
    plt.imshow(x_train[i].numpy().squeeze(), cmap='gray')
    plt.title(f'32x32 패딩된 이미지\n레이블: {np.argmax(y_train[i])}')
    plt.axis('off')
plt.suptitle('전처리된 MNIST 이미지 (32x32)', fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
# LeNet-5 모델 구현
def create_lenet5():
    """
    LeNet-5 아키텍처를 구현하는 함수
    
    원본 LeNet-5 구조:
    - 입력: 32x32x1
    - C1: 6개의 5x5 컨볼루션 (28x28x6)
    - S2: 2x2 평균 풀링 (14x14x6) → MaxPooling 사용
    - C3: 16개의 5x5 컨볼루션 (10x10x16)
    - S4: 2x2 평균 풀링 (5x5x16) → MaxPooling 사용
    - C5: 120개의 5x5 컨볼루션 → Dense(120)으로 구현
    - F6: 84개의 완전연결층
    - 출력: 10개의 클래스
    """
    model = keras.Sequential([
        # 입력층
        layers.Input(shape=(32, 32, 1)),
        
        # C1: 첫 번째 컨볼루션 층 (6개의 5x5 필터)
        layers.Conv2D(filters=6, kernel_size=5, activation='tanh', name='C1'),
        
        # S2: 첫 번째 풀링 층 (2x2 맥스풀링)
        layers.MaxPooling2D(pool_size=2, strides=2, name='S2'),
        
        # C3: 두 번째 컨볼루션 층 (16개의 5x5 필터)
        layers.Conv2D(filters=16, kernel_size=5, activation='tanh', name='C3'),
        
        # S4: 두 번째 풀링 층 (2x2 맥스풀링)
        layers.MaxPooling2D(pool_size=2, strides=2, name='S4'),
        
        # 평탄화
        layers.Flatten(),
        
        # C5: 세 번째 완전연결층 (120개 뉴런)
        layers.Dense(120, activation='tanh', name='C5'),
        
        # F6: 네 번째 완전연결층 (84개 뉴런)
        layers.Dense(84, activation='tanh', name='F6'),
        
        # 출력층: 10개 클래스 (softmax 활성화)
        layers.Dense(10, activation='softmax', name='Output')
    ])
    
    return model

# 모델 생성
print("LeNet-5 모델 생성 중...")
model = create_lenet5()

# 모델 구조 출력
model.summary()


In [ ]:
# 모델 아키텍처 시각화
try:
    keras.utils.plot_model(
        model, 
        to_file='lenet5_architecture.png',
        show_shapes=True, 
        show_layer_names=True,
        rankdir='TB'
    )
    from IPython.display import Image
    Image('lenet5_architecture.png')
except:
    print("모델 시각화를 위해서는 graphviz와 pydot이 필요합니다.")
    print("pip install graphviz pydot 으로 설치할 수 있습니다.")

# 파라미터 수 계산
total_params = model.count_params()
print(f"\n총 파라미터 수: {total_params:,}")

# 각 층별 출력 크기 확인
print("\n각 층별 출력 크기:")
for i, layer in enumerate(model.layers):
    print(f"{i+1}. {layer.name}: {layer.output_shape}")


In [ ]:
# 모델 컴파일
print("모델 컴파일 중...")
model.compile(
    optimizer='adam',  # 원래 LeNet-5는 SGD를 사용했지만, Adam이 더 효율적
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 콜백 함수 정의
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    )
]

print("컴파일 완료!")


In [ ]:
# 모델 훈련
print("모델 훈련 시작...")
print("="*50)

# 훈련 파라미터
EPOCHS = 20
BATCH_SIZE = 128
VALIDATION_SPLIT = 0.1

# 훈련 실행
history = model.fit(
    x_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    callbacks=callbacks,
    verbose=1
)

print("="*50)
print("훈련 완료!")


In [ ]:
# 훈련 과정 시각화
plt.figure(figsize=(15, 5))

# 정확도 그래프
plt.subplot(1, 3, 1)
plt.plot(history.history['accuracy'], label='훈련 정확도', linewidth=2)
plt.plot(history.history['val_accuracy'], label='검증 정확도', linewidth=2)
plt.title('모델 정확도')
plt.xlabel('에포크')
plt.ylabel('정확도')
plt.legend()
plt.grid(True, alpha=0.3)

# 손실 그래프
plt.subplot(1, 3, 2)
plt.plot(history.history['loss'], label='훈련 손실', linewidth=2)
plt.plot(history.history['val_loss'], label='검증 손실', linewidth=2)
plt.title('모델 손실')
plt.xlabel('에포크')
plt.ylabel('손실')
plt.legend()
plt.grid(True, alpha=0.3)

# 학습률 그래프 (있는 경우)
plt.subplot(1, 3, 3)
if 'lr' in history.history:
    plt.plot(history.history['lr'], linewidth=2)
    plt.title('학습률 변화')
    plt.xlabel('에포크')
    plt.ylabel('학습률')
    plt.yscale('log')
    plt.grid(True, alpha=0.3)
else:
    plt.text(0.5, 0.5, '학습률 정보 없음', ha='center', va='center', transform=plt.gca().transAxes)
    plt.title('학습률')

plt.tight_layout()
plt.show()

# 최종 훈련 결과 출력
final_train_acc = max(history.history['accuracy'])
final_val_acc = max(history.history['val_accuracy'])
final_train_loss = min(history.history['loss'])
final_val_loss = min(history.history['val_loss'])

print(f"최고 훈련 정확도: {final_train_acc:.4f}")
print(f"최고 검증 정확도: {final_val_acc:.4f}")
print(f"최소 훈련 손실: {final_train_loss:.4f}")
print(f"최소 검증 손실: {final_val_loss:.4f}")


In [ ]:
# 테스트 데이터로 모델 평가
print("테스트 데이터로 모델 평가 중...")
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)

print(f"테스트 손실: {test_loss:.4f}")
print(f"테스트 정확도: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

# 예측 수행
print("\n예측 수행 중...")
y_pred = model.predict(x_test, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# 분류 보고서
print("\n분류 보고서:")
print(classification_report(y_true_classes, y_pred_classes, 
                          target_names=[str(i) for i in range(10)]))


In [ ]:
# 혼동 행렬 시각화
cm = confusion_matrix(y_true_classes, y_pred_classes)

plt.figure(figsize=(12, 5))

# 혼동 행렬 (숫자)
plt.subplot(1, 2, 1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=range(10), yticklabels=range(10))
plt.title('혼동 행렬 (개수)')
plt.xlabel('예측된 클래스')
plt.ylabel('실제 클래스')

# 혼동 행렬 (비율)
plt.subplot(1, 2, 2)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_normalized, annot=True, fmt='.3f', cmap='Blues',
            xticklabels=range(10), yticklabels=range(10))
plt.title('혼동 행렬 (비율)')
plt.xlabel('예측된 클래스')
plt.ylabel('실제 클래스')

plt.tight_layout()
plt.show()

# 클래스별 정확도
class_accuracy = cm.diagonal() / cm.sum(axis=1)
print("\n클래스별 정확도:")
for i, acc in enumerate(class_accuracy):
    print(f"숫자 {i}: {acc:.4f} ({acc*100:.2f}%)")


In [ ]:
# 예측 결과 시각화
def plot_predictions(images, true_labels, pred_labels, pred_probs, num_samples=16):
    """예측 결과를 시각화하는 함수"""
    plt.figure(figsize=(16, 12))
    
    for i in range(num_samples):
        plt.subplot(4, 4, i + 1)
        
        # 이미지 표시 (패딩 제거)
        img = images[i].squeeze()[2:30, 2:30]  # 32x32에서 28x28로 복원
        plt.imshow(img, cmap='gray')
        
        # 예측 결과 색상 설정
        color = 'green' if true_labels[i] == pred_labels[i] else 'red'
        
        plt.title(f'실제: {true_labels[i]}, 예측: {pred_labels[i]}\n'
                 f'확신도: {pred_probs[i]:.3f}', color=color, fontsize=10)
        plt.axis('off')
    
    plt.suptitle('LeNet-5 예측 결과 (녹색: 정답, 빨강: 오답)', fontsize=16)
    plt.tight_layout()
    plt.show()

# 랜덤 샘플 선택
np.random.seed(42)
sample_indices = np.random.choice(len(x_test), 16, replace=False)

sample_images = x_test[sample_indices]
sample_true = y_true_classes[sample_indices]
sample_pred = y_pred_classes[sample_indices]
sample_probs = np.max(y_pred[sample_indices], axis=1)

plot_predictions(sample_images, sample_true, sample_pred, sample_probs)


In [ ]:
# 오분류 사례 분석
misclassified_indices = np.where(y_true_classes != y_pred_classes)[0]
print(f"총 오분류 개수: {len(misclassified_indices)}")
print(f"오분류율: {len(misclassified_indices)/len(y_test)*100:.2f}%")

# 오분류 사례 중 일부 시각화
if len(misclassified_indices) > 0:
    # 가장 확신도가 높은 오분류 사례들 선택
    misclassified_probs = np.max(y_pred[misclassified_indices], axis=1)
    top_confident_wrong = misclassified_indices[np.argsort(misclassified_probs)[-16:]]
    
    plt.figure(figsize=(16, 12))
    for i, idx in enumerate(top_confident_wrong):
        plt.subplot(4, 4, i + 1)
        
        # 이미지 표시 (패딩 제거)
        img = x_test[idx].squeeze()[2:30, 2:30]
        plt.imshow(img, cmap='gray')
        
        true_label = y_true_classes[idx]
        pred_label = y_pred_classes[idx]
        confidence = np.max(y_pred[idx])
        
        plt.title(f'실제: {true_label}, 예측: {pred_label}\n확신도: {confidence:.3f}', 
                 color='red', fontsize=10)
        plt.axis('off')
    
    plt.suptitle('가장 확신도가 높은 오분류 사례들', fontsize=16)
    plt.tight_layout()
    plt.show()
else:
    print("모든 예측이 정확합니다!")


In [ ]:
# 모델 저장
model_save_path = 'lenet5_mnist_model.keras'
model.save(model_save_path)
print(f"모델이 '{model_save_path}'에 저장되었습니다.")

# 결과 요약
print("\n" + "="*60)
print("LeNet-5 MNIST 분류 모델 결과 요약")
print("="*60)
print(f"모델 아키텍처: LeNet-5 (1998년 Yann LeCun)")
print(f"데이터셋: MNIST (60,000 훈련, 10,000 테스트)")
print(f"입력 크기: 32x32x1 (28x28 MNIST를 패딩)")
print(f"총 파라미터: {total_params:,}")
print(f"훈련 에포크: {len(history.history['accuracy'])}")
print(f"배치 크기: {BATCH_SIZE}")
print(f"옵티마이저: Adam")
print("-"*60)
print(f"최종 테스트 정확도: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"최종 테스트 손실: {test_loss:.4f}")
print(f"총 오분류 개수: {len(misclassified_indices):,}")
print(f"오분류율: {len(misclassified_indices)/len(y_test)*100:.2f}%")
print("="*60)

# 클래스별 성능 요약
print("\n클래스별 성능 요약:")
for i in range(10):
    class_acc = class_accuracy[i]
    class_total = cm.sum(axis=1)[i]
    class_correct = cm[i, i]
    print(f"숫자 {i}: {class_correct:,}/{class_total:,} = {class_acc:.4f} ({class_acc*100:.2f}%)")

print(f"\n모델 훈련 및 평가가 완료되었습니다!")
print(f"LeNet-5는 MNIST 데이터에서 {test_accuracy*100:.2f}%의 정확도를 달성했습니다.")
